# MSc Simple Multi-Objective AutoML Pipeline

First I will look at 2 sets of data, 1 binary classification and one multiclass. I will do a complete pipeline and evaluation that will later be automated. That pipeline will include:

* Data Preprocessing
* Model Training
* Hyperparameter optimization.
* Model Evaluation with Accuracy
* Model Evaluation with Interpretability (specified metrics)
* Incorporating into a ranking function for model selection

Each automation will be constrained.

<img src="images\Pipeline.png"/>

Algorithms that will be used for modelling will be limited to:

1. Logistic Regression
2. Decision Trees
3. Random Forest
4. LightGBM
5. SVM

## Preprocessing Data

This study will assume that datasets input into the automated pipeline will be reasonably clean and representative, as commonly provided by machine learning dataset repositories. Therefore extensive data cleaning and feature engineering are considered outside the scope of this work.

In [1]:
# Importing the libraries needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Import the algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier

Telco Customer Churn:

https://www.kaggle.com/datasets/blastchar/telco-customer-churn 

In [2]:
# Import dataset
churn_data = pd.read_csv("Datasets/Telco-Customer-Churn.csv")
print(type(churn_data))

<class 'pandas.DataFrame'>


In [3]:
# Check the shape of the dataframe
churn_data.shape

(7043, 21)

In [4]:
# Inspect the datatype of each feature and if there are any missing values
churn_data.info()

<class 'pandas.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   str    
 1   gender            7043 non-null   str    
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   str    
 4   Dependents        7043 non-null   str    
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   str    
 7   MultipleLines     7043 non-null   str    
 8   InternetService   7043 non-null   str    
 9   OnlineSecurity    7043 non-null   str    
 10  OnlineBackup      7043 non-null   str    
 11  DeviceProtection  7043 non-null   str    
 12  TechSupport       7043 non-null   str    
 13  StreamingTV       7043 non-null   str    
 14  StreamingMovies   7043 non-null   str    
 15  Contract          7043 non-null   str    
 16  PaperlessBilling  7043 non-null   str    
 17  Paymen

In [5]:
# Summary statistics
churn_data.describe(include='object')

C:\Users\Munro\AppData\Local\Temp\ipykernel_30784\1231571724.py:2: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  churn_data.describe(include='object')


,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,7590-VHVEG,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,20.2,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [6]:
churn_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Quick things to realise when inspecting the data:

* The ID column will be useless in modelling and will need to be dropped consistently accross datasets, it can be seen that the **number of unique values = number of rows** therefore some logic may be implemented to decide when to drop.

* The column "TotalCharges" has 6531 unique values, after inspection, these are numerical charge values stored as strings, in the dtype object. it would be good to process these in the pipeline.

To maintain a simple and generic preprocessing pipeline, features with type object can be treated as categorical variables. It can be implemented that features with more than 20 distinct values can be dropped, as this represents high-cardinality and can negatively impact interpretability and explanation stability. No attempt will be made to coerce these features to numeric values, as this would require dataset specific cleaning logic and would defeat the point of implementing this across datasets.

In [7]:
# creating a function for identifying data types of each column for preprocessing, splitting them into groups

def get_feature_groups(data):

    categorical_columns = data.select_dtypes(include=["object", "string", "category"]).columns #detect all categorical column labels
    numeric_columns = data.select_dtypes(include="number").columns #detect all numerical column labels 
    bool_columns = data.select_dtypes(include="bool").columns #detect possible boolean columns

    numeric_columns = numeric_columns.union(bool_columns).tolist() #merge boolean in with numeric as they are 0 and 1 representations

    high_cardinality = data[categorical_columns].nunique() > 20 #isolate high cardinality features

    #columns_to_drop = high_cardinality[high_cardinality].index.tolist() #create a column to drop list
    categorical_columns = high_cardinality[~high_cardinality].index.tolist() #dropping high cardinality features from categorical list
    
    return numeric_columns, categorical_columns, #columns_to_drop


By creating a list for each type of variable, this will lead into transforming the columns and preparing for a scikit learn pipeline.

In [8]:
# Split the data into training and test sets
y = churn_data["Churn"]
X = churn_data.drop("Churn", axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape


((5634, 20), (5634,), (1409, 20), (1409,))

In [9]:
numerical_columns, categorical_columns = get_feature_groups(X_train)

In [10]:
print(numerical_columns)
print(categorical_columns)

['SeniorCitizen', 'tenure', 'MonthlyCharges']
['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


In [11]:
# Create a column transformer

preprocesser = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_columns),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_columns)
    ],
    remainder = "drop",
    n_jobs=-1 # enable use of all cpu threads locally
)

Column transformer will help to transform the numerical features and the categorical features.

In [12]:
# Create a dictionary of models to iterate over

models = {
    "logreg": LogisticRegression(),
    "dt": DecisionTreeClassifier(),
    "rf": RandomForestClassifier(),
    "svm": LinearSVC(),
    "lgb": LGBMClassifier(),
}

In [13]:
# Create an iterative pipeline

results = {}

for name, model in models.items():
    pipeline = Pipeline([
        ("preprocess", preprocesser),
        ("model", model)
    ])

    pipeline.fit(X_train, y_train)
    score = pipeline.score(X_test, y_test)

    results[name] = score

[LightGBM] [Info] Number of positive: 1496, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000499 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 414
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265531 -> initscore=-1.017418
[LightGBM] [Info] Start training from score -1.017418


c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [14]:
print(results)

{'logreg': 0.8211497515968772, 'dt': 0.7139815471965933, 'rf': 0.78708303761533, 'svm': 0.8183108587650816, 'lgb': 0.7984386089425124}


While the previous pipeline it clean, if cross validation is used feature selection based on X_train will cause data leakage for each fold as the validation set would have been used to make transformations on the training data. Hence we want to implement feature selection and preprocessing of data for each training fold.

In [15]:
# Split the data into training and test sets

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = le.fit_transform(churn_data["Churn"]) # encode category labels
X = churn_data.drop("Churn", axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5634, 20), (5634,), (1409, 20), (1409,))

In [16]:
# Create a transformer class to group features in first step of a pipeline

from sklearn.base import BaseEstimator, TransformerMixin

class GroupFeatures(BaseEstimator, TransformerMixin):

    # Constructor method
    def __init__(self, max_unique=20):
        self.max_unique = max_unique

    #Fit method learns column groupings and detects high cardinality --> metadata for future steps
    def fit(self, X, y=None):
        categorical = X.select_dtypes(include=["object", "string", "category"])
        numeric = X.select_dtypes(include="number")
        boolean = X.select_dtypes(include="bool")

        high_cardinality = categorical.nunique() > self.max_unique #Detects high cardinality above maximum value specified

        self.numeric_columns_ = numeric.columns.union(boolean.columns).tolist() #Creates numerical column list
        self.categorical_columns_ = high_cardinality[~high_cardinality].index.tolist() #Creates categorical column list - high cardinality features

        self.keep_columns_ = self.numeric_columns_ + self.categorical_columns_

        return self

    # passes through the data untransformed 
    def transform(self, X):
        return X[self.keep_columns_]

In [17]:
# ColumnTransformer for feature preprocessing
from sklearn.compose import make_column_selector

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), make_column_selector(dtype_include="number")), 
        ("cat", OneHotEncoder(handle_unknown="ignore"), make_column_selector(dtype_include=["object", "string", "category"])),
    ],
    remainder="drop"
)

In [18]:
# Implement Cross-Validation Evaluation

from sklearn.model_selection import StratifiedKFold, cross_validate

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    "accuracy": "accuracy",
    "f1": "f1",
    "roc_auc": "roc_auc"
}

for name, model in models.items():

    # Create pipeline that include GroupFeatures in first step
    pipeline = Pipeline([
    ("features", GroupFeatures(max_unique=20)),
    ("preprocess", preprocessor),
    ("model", model)
    ])

    cv_results = cross_validate(
    pipeline,
    X_train,
    y_train,
    cv=cv,
    scoring=scoring,
    return_estimator=True
    )

    results[name] = cv_results

[LightGBM] [Info] Number of positive: 1197, number of negative: 3310
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000271 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 414
[LightGBM] [Info] Number of data points in the train set: 4507, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265587 -> initscore=-1.017130
[LightGBM] [Info] Start training from score -1.017130
[LightGBM] [Info] Number of positive: 1197, number of negative: 3310
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000378 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 414
[LightGBM] [Info] Number of data points in the train set: 4507, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265587 -> initscore=-1.017130
[LightGBM]

c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 1197, number of negative: 3310
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000443 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 414
[LightGBM] [Info] Number of data points in the train set: 4507, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265587 -> initscore=-1.017130
[LightGBM] [Info] Start training from score -1.017130
[LightGBM] [Info] Number of positive: 1196, number of negative: 3311
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000387 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 414
[LightGBM] [Info] Number of data points in the train set: 4507, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265365 -> initscore=-1.018268
[LightGBM] [Info] Start training from score -1.018268


c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 1197, number of negative: 3311
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000289 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 414
[LightGBM] [Info] Number of data points in the train set: 4508, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265528 -> initscore=-1.017432
[LightGBM] [Info] Start training from score -1.017432


c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [19]:
results['logreg']['test_accuracy']

array([0.78881988, 0.81011535, 0.78615794, 0.80834073, 0.79928952])

In [20]:
results['logreg']

{'fit_time': array([0.0511601 , 0.04904747, 0.04824543, 0.04755926, 0.04179358]),
 'score_time': array([0.02668858, 0.02503681, 0.02410722, 0.0256021 , 0.02678967]),
 'estimator': [Pipeline(steps=[('features', GroupFeatures()),
                  ('preprocess',
                   ColumnTransformer(transformers=[('num', StandardScaler(),
                                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000020869246ED0>),
                                                   ('cat',
                                                    OneHotEncoder(handle_unknown='ignore'),
                                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000002086DB7BED0>)])),
                  ('model', LogisticRegression())]),
  Pipeline(steps=[('features', GroupFeatures()),
                  ('preprocess',
                   ColumnTransformer(transformers=[('num', StandardScaler(),
            

`RandomSearchCV()` will be used for hyperparameter optimisation and a parameter space needs to be defined, for example:

Logistic Regression → C, penalty

Decision Tree → max_depth, min_samples_leaf

Random Forest → n_estimators, max_depth

XGBoost/LightGBM → max_depth, learning_rate, n_estimators

In [21]:
# Define param distributions

from scipy.stats import loguniform, randint, uniform

param_grids = {
    "logreg" : [ #Not all solvers are compatible with each type of regularization. so an list of dictionaries is implemented to enforce compatibility.
        {   # liblinear: supports l1, l2
            "model__solver": ["liblinear"],
            "model__penalty": ["l1", "l2"],
            "model__C": loguniform(1e-4, 1e2),
            "model__class_weight": [None, "balanced"],
        },
        {   # lbfgs: supports l2
            "model__solver": ["lbfgs"],
            "model__penalty": ["l2"],
            "model__C": loguniform(1e-4, 1e2),
            "model__class_weight": [None, "balanced"],
        },
        {   # saga: supports elasticnet (needs l1_ratio)
            "model__solver": ["saga"],
            "model__penalty": ["elasticnet"],
            "model__C": loguniform(1e-4, 1e2),
            "model__l1_ratio": [0.1, 0.5, 0.9], #only used for elasticnet
            "model__class_weight": [None, "balanced"],
        },
        {   # saga can do l1 / l2 as well
            "model__solver": ["saga"],
            "model__penalty": ["l1", "l2"],
            "model__C": loguniform(1e-4, 1e2),
            "model__class_weight": [None, "balanced"],
        },
        ],

    "dt" : {
        "model__max_depth": randint(1, 20), 
        "model__min_samples_split": randint(2, 20), 
        "model__min_samples_leaf": randint(1, 20),
        "model__class_weight": [None, "balanced"],
    },
    "rf" : {
        "model__n_estimators": randint(100, 500),
        "model__max_depth": randint(1, 20),
        "model__min_samples_split": randint(2, 20),
        "model__class_weight": [None, "balanced", "balanced_subsample"],
    },
    "svm" : [
        {
            "model__penalty": ["l2"],
            "model__loss": ["squared_hinge", "hinge"],
            "model__C": loguniform(1e-4, 1e2),
            "model__class_weight": [None, "balanced"],
        },
        { #L1 only works with square_hinge and model_dual = False --> model_dual is defaulted as "auto"
            "model__penalty": ["l1"],
            "model__loss": ["squared_hinge"],
            "model__C": loguniform(1e-4, 1e2),
            "model__class_weight": [None, "balanced"],
        },
        ],
    "lgb" : {
        "model__num_leaves": randint(31, 60),       # controls tree complexity
        "model__max_depth": randint(5, 15),         # limits tree depth
        "model__learning_rate": loguniform(1e-3, 0.1), # step size
        "model__n_estimators": randint(100, 500),   # number of boosting rounds
        "model__subsample": uniform(0.7, 0.3),      # row sampling
        "model__colsample_bytree": uniform(0.7, 0.3), # feature sampling
        "model__class_weight": [None, "balanced"], # to handle class imbalance
    }
}

In [22]:
# Implement RandomSearchCV test wutg 5 iterations

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.model_selection import RandomizedSearchCV

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    "accuracy": "accuracy",
    "f1": "f1",
    "roc_auc": "roc_auc"
}

for name, model in models.items():

    # Create pipeline that include GroupFeatures in first step
    pipeline = Pipeline([
    ("features", GroupFeatures(max_unique=20)),
    ("preprocess", preprocessor),
    ("model", model)
    ])

    clf = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_grids[name],
        n_iter=5,
        scoring=scoring,
        cv=cv,
        n_jobs=-1,
        random_state=42,
        refit="roc_auc" #Refit an estimator using the best found parameters on the whole dataset using one main metric
    )

    search = clf.fit(X_train, y_train)

    results[name] = search

    best_idx = search.best_index_

    print("\n==============================")
    print(f"MODEL: {name}")
    print("==============================")
    print("Best Params:", search.best_params_)
    print(f"Best ROC-AUC:  {search.cv_results_['mean_test_roc_auc'][best_idx]:.4f}")
    print(f"Best Accuracy: {search.cv_results_['mean_test_accuracy'][best_idx]:.4f}")
    print(f"Best F1:       {search.cv_results_['mean_test_f1'][best_idx]:.4f}")



c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



MODEL: logreg
Best Params: {'model__C': np.float64(0.39079671568228835), 'model__class_weight': None, 'model__penalty': 'l2', 'model__solver': 'saga'}
Best ROC-AUC:  0.8385
Best Accuracy: 0.7984
Best F1:       0.5824

MODEL: dt
Best Params: {'model__class_weight': None, 'model__max_depth': 4, 'model__min_samples_leaf': 8, 'model__min_samples_split': 4}
Best ROC-AUC:  0.8231
Best Accuracy: 0.7865
Best F1:       0.5170

MODEL: rf
Best Params: {'model__class_weight': None, 'model__max_depth': 7, 'model__min_samples_split': 12, 'model__n_estimators': 187}
Best ROC-AUC:  0.8380
Best Accuracy: 0.7984
Best F1:       0.5558

MODEL: svm
Best Params: {'model__C': np.float64(0.39079671568228835), 'model__class_weight': None, 'model__loss': 'squared_hinge', 'model__penalty': 'l1'}
Best ROC-AUC:  0.8370
Best Accuracy: 0.7985
Best F1:       0.5780
[LightGBM] [Info] Number of positive: 1496, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing wa

Automatic training and output of each models metrics have been completer, now onto shap values and model ranking.

## SHAP analysis

In [23]:
import shap

c:\Users\Munro\anaconda3\envs\msc_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# get the best logreg model trained on full test set with best parameters
best_logreg_pipeline = results["logreg"].best_estimator_

# the transformed data
X_trans = best_logreg_pipeline.named_steps["preprocess"].transform(X_train)

# The final fitted model
logreg_model = best_logreg_pipeline.named_steps["model"]

In [25]:
print(X_trans)

[[-4.37749204e-01 -4.65683364e-01 -4.73723375e-04 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [-4.37749204e-01  8.85536787e-01  1.07475386e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-4.37749204e-01 -1.28460467e+00 -1.37649913e+00 ...  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 ...
 [-4.37749204e-01 -8.34197950e-01 -1.45294499e+00 ...  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [ 2.28441306e+00 -8.34197950e-01  1.14953785e+00 ...  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [-4.37749204e-01 -2.60953038e-01 -1.49781538e+00 ...  1.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [30]:
print(logreg_model)

LogisticRegression(C=np.float64(0.39079671568228835), penalty='l2',
                   solver='saga')


In [29]:
print(best_logreg_pipeline)

Pipeline(steps=[('features', GroupFeatures()),
                ('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002086FE1D210>),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002087045DC90>)])),
                ('model',
                 LogisticRegression(C=np.float64(0.39079671568228835),
                                    penalty='l2', solver='saga'))])


In [31]:
logreg_pred = best_logreg_pipeline.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

acc = accuracy_score(y_test, logreg_pred)
f1 = f1_score(y_test, logreg_pred)
auc = roc_auc_score(y_test, logreg_pred)

print(acc, f1, auc)

0.8204400283889283 0.6328011611030478 0.7449279037750887
